### Homework 2

Import necessary packages

In [1]:
import pandas as pd
import numpy as np
import wrds
from pandas.tseries.offsets import *
import matplotlib.pyplot as plt

d:\Applications\Anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Import data from wrds

In [2]:


# import wrds
conn = wrds.Connection(wrds_username='YOUR_ID_HERE')

# LOADING CRSP DATA FROM WRDS --  Bonds data
print('Load Bond Data')
bonds = conn.raw_sql("""
                      select kycrspid, mcaldt, tmretnua, tmtotout
                      from crspq.tfz_mth
                      """)
bonds['mcaldt']   = pd.DataFrame(bonds[['mcaldt']].values.astype('datetime64[ns]')) + MonthEnd(0)
bonds = bonds.rename(columns={"mcaldt":"date","tmretnua":"ret","tmtotout":"me","kycrspid":"idCRSP"}).copy()

# LOADING CRSP DATA FROM WRDS -- T-Bill data
print('Load T-Bill Data')
rf = conn.raw_sql("""
                      select caldt, t30ret, t90ret
                      from crspq.mcti
                      """)
rf['caldt']   = pd.DataFrame(rf[['caldt']].values.astype('datetime64[ns]')) + MonthEnd(0)
rf = rf.rename(columns={"caldt":"date","t30ret":"rf30","t90ret":"rf90"}).copy()

# LOADING CRSP DATA FROM WRDS -- Equity Returns
print('Load Equity Returns Data')
crsp_raw = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc, a.cfacshr
                      from crspq.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1900' and '12/31/2024'
                      """)
crsp_raw[['permno', 'permco']] = crsp_raw[['permno', 'permco']].astype(int)
crsp_raw['date']   = pd.to_datetime(crsp_raw['date'],format='%Y-%m-%d',errors='ignore')
crsp_raw['date']   = pd.DataFrame(crsp_raw[['date']].values.astype('datetime64[ns]')) + MonthEnd(0)

# LOADING CRSP DATA FROM WRDS -- Deilsting Returns
print('Load Delisting Returns Data')
dlret_raw = conn.raw_sql("""
                     select permno, dlret, dlstdt, dlstcd
                     from crspq.msedelist
                     """)
dlret_raw.permno    = dlret_raw.permno.astype(int)
dlret_raw['dlstdt'] = pd.to_datetime(dlret_raw['dlstdt'])
dlret_raw['date']  = dlret_raw['dlstdt']+MonthEnd(0)

# LOADING CRSP DATA FROM WRDS -- CRSP VW market index (for comparison purposes)
print('Load VW CRSP market index Data')
mkt_csrp = conn.raw_sql("""
                      select date, VWRETD, totval
                      from crspq.msi
                      """)
mkt_csrp['date']   = pd.DataFrame(mkt_csrp[['date']].values.astype('datetime64[ns]')) + MonthEnd(0)
mkt_csrp = mkt_csrp.rename(columns={"vwretd":"mkt_crsp","totval":"mkt_crsp_mktcap"}).copy()

# Save data
# mkt_csrp.to_pickle(data_folder+'mkt_csrp_PS2.pkl')
# crsp_raw.to_pickle(data_folder+'crsp_raw_PS2.pkl')
# dlret_raw.to_pickle(data_folder+'dlret_raw_PS2.pkl')
# rf.to_pickle(data_folder+'rf_PS2.pkl')
# bonds.to_pickle(data_folder+'bonds_PS2.pkl')

# Close connection to WRDS
conn.close()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Load Bond Data
Load T-Bill Data
Load Equity Returns Data


C:\Users\14561\AppData\Local\Temp\ipykernel_21960\3078274063.py:35: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  crsp_raw['date']   = pd.to_datetime(crsp_raw['date'],format='%Y-%m-%d',errors='ignore')


Load Delisting Returns Data
Load VW CRSP market index Data


In [3]:
CRSP_Bonds = bonds.copy() #copy the data so that data cleaning will not affect original data 
CRSP_Bonds

,idCRSP,date,ret,me
0,19610622.800000,1960-10-31,NaN,3501.0
1,19610622.800000,1960-11-30,0.001631,3503.0
2,19610622.800000,1960-12-31,0.004451,3503.0
3,19610622.800000,1961-01-31,0.001999,3503.0
4,19610622.800000,1961-02-28,0.001277,3503.0
...,...,...,...,...
186256,20241226.400000,2023-12-31,NaN,45518.0
186257,20251231.204250,2023-12-31,NaN,NaN
186258,20261215.204370,2023-12-31,NaN,49999.0
186259,20281231.203750,2023-12-31,NaN,NaN


In [4]:
CRSP_Bonds.set_index('date')

,idCRSP,ret,me
date,,,
1960-10-31,19610622.800000,NaN,3501.0
1960-11-30,19610622.800000,0.001631,3503.0
1960-12-31,19610622.800000,0.004451,3503.0
1961-01-31,19610622.800000,0.001999,3503.0
1961-02-28,19610622.800000,0.001277,3503.0
...,...,...,...
2023-12-31,20241226.400000,NaN,45518.0
2023-12-31,20251231.204250,NaN,NaN
2023-12-31,20261215.204370,NaN,49999.0


#### Question 1

Q: Construct the equal-weighted bond market return, value-weighted bond market return, and
lagged total bond market capitalization using CRSP Bond data 1. Your output should be from January 1926 to December 2023, at a monthly frequency.

Basic Logic of the function:

- convert data column to pd.datetime in order to get year and month later
- calculate equal weighted return by taking means after grouping by date
- calculate value weighted return by $volumn\cdot return/total\space volumn$
- calculate the total ME of a day by grouping 
- format the output
- I tried to directly use the ME but the number look weird as the ME of bonds are 1000 times larger than the stock ME. I referred to google and found that the ME here should be divided by 1e3 which is counterintuitive

In [5]:
def PS2_Q1(df):
    # convert data type
    df['date'] = pd.to_datetime(df['date']) 
    # calculate lag me, according to the data on google, inorder to compare with the stock data, the me should be divided by 1e3
    df['lag_me'] = df.groupby('idCRSP')['me'].shift(1)/1e3 
    df = df.dropna()
    
    
    # for each month, calculate equal weighted return 
    ew_ret = df.groupby('date')['ret'].mean() 
    
    #  calculate vaue weighted return
    df['vw_ret'] = (df['ret']*df['lag_me']).copy()
    vw_ret = df.groupby('date')['vw_ret'].sum()/df.groupby('date')['lag_me'].sum()
    
    # calculate total me of a day
    me = df.groupby('date')['lag_me'].sum()
    
    # export dataser
    return_df = pd.DataFrame({"Year":ew_ret.index.year,
                              'Month':ew_ret.index.month,
                              "Bond_Lag_MV":me,
                              "Bond_Ew_Ret":ew_ret,
                              "Bond_Vw_Ret":vw_ret})
    
    return_df = return_df.reset_index()
    
    return return_df
Monthly_CRSP_Bonds = PS2_Q1(CRSP_Bonds)
Monthly_CRSP_Bonds


C:\Users\14561\AppData\Local\Temp\ipykernel_21960\2874764197.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vw_ret'] = (df['ret']*df['lag_me']).copy()


,date,Year,Month,Bond_Lag_MV,Bond_Ew_Ret,Bond_Vw_Ret
0,1926-01-31,1926,1,19.502,0.005101,0.006829
1,1926-02-28,1926,2,19.502,0.003621,0.003844
2,1926-03-31,1926,3,18.884,0.003812,0.003588
3,1926-04-30,1926,4,19.231,0.004014,0.006486
4,1926-05-31,1926,5,19.227,0.002146,0.002629
...,...,...,...,...,...,...
1171,2023-08-31,2023,8,20790.095,-0.003801,-0.003940
1172,2023-09-30,2023,9,21376.899,-0.017292,-0.017146
1173,2023-10-31,2023,10,21114.709,-0.011088,-0.011151
1174,2023-11-30,2023,11,21274.092,0.032737,0.032877


#### Question 2

Q: Aggregate stock, bond, and riskless datatables. For each year-month, calculate the lagged
market value and excess value-weighted returns for both stocks and bonds. Your output should
be from January 1926 to December 2023, at a monthly frequency.

Load stock and RF data

In [6]:
Monthly_CRSP_Riskless = rf.copy()

Cleaning the stock and RF data

- load csv file from PS1
- convert data in RF dataframe into year and month for merging
- rename the columns to make it really clear
- delete  the first row of RF data as it is in 1925

In [7]:
Monthly_CRSP_Stocks = pd.read_csv('Stock_CRSP.csv')
# construct year and monnth
Monthly_CRSP_Riskless['date'] = pd.to_datetime(Monthly_CRSP_Riskless['date'])
Monthly_CRSP_Riskless['Year'] = Monthly_CRSP_Riskless['date'].dt.year
Monthly_CRSP_Riskless['Month'] = Monthly_CRSP_Riskless['date'].dt.month
# drop date in the original df
Monthly_CRSP_Bonds.drop('date',axis = 1,inplace=True)
Monthly_CRSP_Stocks.drop('date',axis = 1,inplace=True)
Monthly_CRSP_Riskless.drop('date',axis = 1,inplace=True)
# rename column
Monthly_CRSP_Stocks.columns = ['Year', 'Month', 'Stock_Lag_MV', 'Stock_Ew_Ret',
       'Stock_Vw_Ret']
Monthly_CRSP_Riskless.columns=["RF_30","RF_90",'Year', 'Month']
# drop first row
Monthly_CRSP_Riskless = Monthly_CRSP_Riskless[1:]
Monthly_CRSP_Riskless


,RF_30,RF_90,Year,Month
1,0.002951,0.004042,1926,1
2,0.002768,0.003237,1926,2
3,0.002778,0.002778,1926,3
4,0.003072,0.003072,1926,4
5,0.000342,0.001965,1926,5
...,...,...,...,...
1172,0.004537,0.003365,2023,8
1173,0.004267,0.004288,2023,9
1174,0.004715,0.004777,2023,10
1175,0.004424,0.004597,2023,11


Function of Q2

 - merge the stock and RF data to the bonds data according to year and month
 - construct excess return by return-RF rate
 - I am using RF_30 as the riskfree rate here as it is monthly return so it makkes sense to use RF_30

In [8]:
def PS2_Q2(B, S, R):
    df = pd.merge(B,S,on=['Year','Month'],how = 'left')
    df = pd.merge(df,R,on=['Year','Month'],how = 'left')
    df['Stock_Excess_Vw_Ret'] = df['Stock_Vw_Ret']-df['RF_30']
    df['Bond_Excess_Vw_Ret'] = df['Bond_Vw_Ret']-df['RF_30']
    return df[['Year',"Month","Stock_Lag_MV","Stock_Excess_Vw_Ret","Bond_Lag_MV","Bond_Excess_Vw_Ret"]]
    
Monthly_CRSP_Universe = PS2_Q2(Monthly_CRSP_Bonds,Monthly_CRSP_Stocks,Monthly_CRSP_Riskless)
Monthly_CRSP_Universe

,Year,Month,Stock_Lag_MV,Stock_Excess_Vw_Ret,Bond_Lag_MV,Bond_Excess_Vw_Ret
0,1926,1,0.000000,NaN,19.502,0.003878
1,1926,2,27.032250,-0.036744,19.502,0.001076
2,1926,3,26.128629,-0.067653,18.884,0.000810
3,1926,4,24.506932,0.033663,19.231,0.003414
4,1926,5,25.178699,0.011629,19.227,0.002287
...,...,...,...,...,...,...
1171,2023,8,45247.097359,-0.023875,20790.095,-0.008477
1172,2023,9,44202.187878,-0.052440,21376.899,-0.021413
1173,2023,10,41958.286962,-0.031872,21114.709,-0.015866
1174,2023,11,40654.841180,0.088365,21274.092,0.028453


#### Question 3

Q: Calculate the monthly unlevered and levered risk-parity portfolio returns as defined by Asness,
Frazzini, and Pedersen (2012).3 For the levered risk-parity portfolio, match the value-weighted
portfolio’s ˆσ over the longest matched holding period of both. Your output should be from January
1926 to December 2023, at a monthly frequency.

In [9]:
Monthly_CRSP_Universe_3 = Monthly_CRSP_Universe.copy()

Logic of the function
 - In order to compare the result of my function with the sample given by professor and the paper, I restrict time between 1929.1-2010.6
 - the Excess_Vw_Ret is the value weighted return combining stock and bond weighted by their ME. 
 - the Excess_60_40_Ret is a portfolio consisting 60% of Stock and 40% of bond. There is a question here: consider the actual case. If you choose to rebalance once a month, you can only balance this month's portfolio according to last month's ME, so it is impossible to get exactly 60% of stock return and 40% of bonds return. However I assume it will not make much difference so I ignore the problem here
 - the Sigma_Hat column for Stocks and Bonds are constructed according to the essay, by taking prior 3 year's volatility as an estimator for the volatility.
 - Also, according to the paper, unleveraged k is constructed according as $\frac{1}{\sigma_{Stock}^{-1}+\sigma_{Bond}^{-1}}$
 - Then, weight is calculated as $w_i = k_i\cdot\sigma_i^{-1}$ 
 - After that we calculate the return by $r_{portfolio} = \sum _i w_ir_i$
 - The way of constructing leveraged k is arbitrage. However, the paper chose k such that the volatility of the portfolio matched the value weighted portfolio calculated before. The paper says that the leveraged k is a constant. So I calculated k by $k = \frac{target\space\sigma} {unleveraged\space\sigma} $
 - by constructing the leveraged k this way, the $sigma_{leveraged} = sigma_{value weighted}$
 - however, by taking a constant k, the skewness and kurtosis is the same for leveraged and unleveraged portfolio. According to the table in the paper, they should be different 
 - I tried to another way of construcitng k by matching the volatility from month to month, However, the total volatility of the two portfolio does not match so I return to the original method
 - I am not sure how does the paper construct the  leveraged k 
 - The method of calculating leveraged portfolio is  the same as unleveraged portfolio


In [10]:
def PS2_Q3(df):
    df['Vw_Stock_Weight'] = df['Stock_Lag_MV']/(df['Stock_Lag_MV']+df['Bond_Lag_MV'])
    df['Excess_Vw_Ret'] = df['Vw_Stock_Weight'] *df['Stock_Excess_Vw_Ret']+(1-df['Vw_Stock_Weight'] )*df['Bond_Excess_Vw_Ret']

    df['Excess_60_40_Ret'] = 0.6*df['Stock_Excess_Vw_Ret']+0.4*df['Bond_Excess_Vw_Ret']
    
    # sigma for bond and stock
    df['Stock_Sigma_Hat'] = df['Stock_Excess_Vw_Ret'].rolling(window = 36,closed = 'left').std()
    df['Bond_Sigma_Hat'] = df['Bond_Excess_Vw_Ret'].rolling(window = 36,closed = 'left').std()
    # get its inverse 
    df['Stock_Inverse_Sigma_Hat'] = 1/df['Stock_Sigma_Hat'] 
    df['Bond_Inverse_Sigma_Hat'] = 1/df['Bond_Sigma_Hat']
    # unleveraged k
    df['Unleveraged_K'] = 1/(df['Stock_Inverse_Sigma_Hat']+df['Bond_Inverse_Sigma_Hat'])
    # stock weight
    df['RP_Unleveraged_Stock_Weight'] =   df['Unleveraged_K']*df['Stock_Inverse_Sigma_Hat'] 
    # bond weight
    df['RP_Unleveraged_Bond_Weight'] =   df['Unleveraged_K']*df['Bond_Inverse_Sigma_Hat'] 
    # unleveraged RP
    df['Excess_Unleveraged_RP_Ret'] =  (df['RP_Unleveraged_Stock_Weight']*df['Stock_Excess_Vw_Ret']+df['RP_Unleveraged_Bond_Weight']*df['Bond_Excess_Vw_Ret'])
    # df to calculate sigma from 1929-2010
    df_c = df.copy()
    df_c = df_c[df_c['Year']>1928]
    df_c = df_c[df_c['Year']<2011]
    df_c = df_c[:-6]
    # leveraged k
    Sigma_Benchmark = np.std(df_c['Excess_Vw_Ret'])*np.sqrt(12)
    Sigma_unleveraged = np.std(df_c['Excess_Unleveraged_RP_Ret'])*np.sqrt(12)
    # keep the sigma of leveraged RP to be the same as benchmark
    df['Leveraged_K'] = Sigma_Benchmark / Sigma_unleveraged

    # now calculate the leveraged return
    df['Excess_Leveraged_RP_Ret'] = df['Leveraged_K']*(df['RP_Unleveraged_Stock_Weight']*df['Stock_Excess_Vw_Ret']+df['RP_Unleveraged_Bond_Weight']*df['Bond_Excess_Vw_Ret'])
    
    df = df[df['Year']>1928]
    df = df[df['Year']<2011]
    df = df[:-6]
    return df[['Year', 'Month', 'Stock_Excess_Vw_Ret', 
       'Bond_Excess_Vw_Ret',  'Excess_Vw_Ret',
       'Excess_60_40_Ret',
       'Stock_Inverse_Sigma_Hat', 'Bond_Inverse_Sigma_Hat', 'Unleveraged_K',
       'Excess_Unleveraged_RP_Ret', 'Leveraged_K', 'Excess_Leveraged_RP_Ret']]
Port_Rets = PS2_Q3(Monthly_CRSP_Universe_3)
Port_Rets

,Year,Month,Stock_Excess_Vw_Ret,Bond_Excess_Vw_Ret,Excess_Vw_Ret,Excess_60_40_Ret,Stock_Inverse_Sigma_Hat,Bond_Inverse_Sigma_Hat,Unleveraged_K,Excess_Unleveraged_RP_Ret,Leveraged_K,Excess_Leveraged_RP_Ret
36,1929,1,0.046589,-0.004701,0.035325,0.026073,NaN,332.034333,NaN,NaN,3.237856,NaN
37,1929,2,-0.003324,-0.004926,-0.003660,-0.003965,25.380295,328.043165,0.002829,-0.004811,3.237856,-0.015576
38,1929,3,-0.008841,-0.007900,-0.008648,-0.008465,26.020542,317.860119,0.002908,-0.007972,3.237856,-0.025811
39,1929,4,0.014252,0.008770,0.013100,0.012059,27.965521,295.137306,0.003095,0.009244,3.237856,0.029931
40,1929,5,-0.063653,-0.010162,-0.052758,-0.042257,27.997745,272.462217,0.003328,-0.015146,3.237856,-0.049042
...,...,...,...,...,...,...,...,...,...,...,...,...
1009,2010,2,0.034023,0.001112,0.022853,0.020859,17.131552,98.214836,0.008670,0.006000,3.237856,0.019428
1010,2010,3,0.063146,-0.004920,0.040148,0.035919,17.039682,98.489739,0.008656,0.005119,3.237856,0.016574
1011,2010,4,0.019962,0.007491,0.015822,0.014973,16.742949,98.084365,0.008709,0.009309,3.237856,0.030141
1012,2010,5,-0.078950,0.011680,-0.048567,-0.042698,16.803716,97.813324,0.008725,-0.001607,3.237856,-0.005204


#### Question 4 

Q: Replicate and report Panel A of Table 2 in Asness, Frazzini, and Pedersen (2012), except
for Alpha and t-stat of Alpha columns. Specifically, for all strategies considered, report the
annualized average excess returns, t-statistic of the average excess returns, annualized volatility,
annualized Sharpe Ratio, skewness, and excess kurtosis. Your sample should be from January
1929 to June 2010, at monthly frequency. Match the format of the table to the extent possible.
Discuss the difference between your table and the table reported in the paper. It is zero? If
not, justify whether the difference is economically negligible or not. What are the reasons for a
nonzero difference?


In [12]:
def mean(df):
    return np.round(np.mean(df)*1200,2)
def std(df):
    return np.round(np.std(df)*np.sqrt(12)*100,2)
def t_stats(df):
    return np.round((np.mean(df)*12 )/(np.std(df)*np.sqrt(12)/(np.sqrt(len(df)/12))),2)
def sharpe(df):
    return np.round((np.mean(df)*12)/(np.std(df)*np.sqrt(12)),2)
def skewness(df):
    return np.round(df.skew(),2)
def kur(df):
    return np.round(df.kurtosis(),2)

def PS2_Q4(df):
    result = pd.DataFrame()

    result.loc['Annualized Mean','CRSP stocks'] = mean(df['Stock_Excess_Vw_Ret'])
    result.loc['Annualized Mean','CRSP bonds'] = mean(df['Bond_Excess_Vw_Ret'])
    result.loc['Annualized Mean','Value-weighted portfolio'] = mean(df['Excess_Vw_Ret'])
    result.loc['Annualized Mean','60/40 portfolio'] = mean(df['Excess_60_40_Ret'])
    result.loc['Annualized Mean','Unlevered RP'] =mean(df['Excess_Unleveraged_RP_Ret'])
    result.loc['Annualized Mean','Levered RP'] = mean(df['Excess_Leveraged_RP_Ret'])
    result.loc['Annualized Mean','RP minus value weighted'] = mean(df['Excess_Leveraged_RP_Ret']-df['Excess_Vw_Ret'])
    result.loc['Annualized Mean','RP minus 60/40'] = mean(df['Excess_Leveraged_RP_Ret']-df['Excess_60_40_Ret'])
    
    result.loc['t-stat','CRSP stocks'] = t_stats(df['Stock_Excess_Vw_Ret'])
    result.loc['t-stat','CRSP bonds'] = t_stats(df['Bond_Excess_Vw_Ret'])
    result.loc['t-stat','Value-weighted portfolio'] = t_stats(df['Excess_Vw_Ret'])
    result.loc['t-stat','60/40 portfolio'] = t_stats(df['Excess_60_40_Ret'])
    result.loc['t-stat','Unlevered RP'] =t_stats(df['Excess_Unleveraged_RP_Ret'])
    result.loc['t-stat','Levered RP'] = t_stats(df['Excess_Leveraged_RP_Ret'])
    result.loc['t-stat','RP minus value weighted'] = t_stats(df['Excess_Leveraged_RP_Ret']-df['Excess_Vw_Ret'])
    result.loc['t-stat','RP minus 60/40'] = t_stats(df['Excess_Leveraged_RP_Ret']-df['Excess_60_40_Ret'])
    
    
    result.loc['Annualized Standard Deviation','CRSP stocks'] = std(df['Stock_Excess_Vw_Ret'])
    result.loc['Annualized Standard Deviation','CRSP bonds'] = std(df['Bond_Excess_Vw_Ret'])
    result.loc['Annualized Standard Deviation','Value-weighted portfolio'] = std(df['Excess_Vw_Ret'])
    result.loc['Annualized Standard Deviation','60/40 portfolio'] = std(df['Excess_60_40_Ret'])
    result.loc['Annualized Standard Deviation','Unlevered RP'] = std(df['Excess_Unleveraged_RP_Ret'])
    result.loc['Annualized Standard Deviation','Levered RP'] = std(df['Excess_Leveraged_RP_Ret'])
    result.loc['Annualized Standard Deviation','RP minus value weighted'] = std(df['Excess_Leveraged_RP_Ret']-df['Excess_Vw_Ret'])
    result.loc['Annualized Standard Deviation','RP minus 60/40'] = std(df['Excess_Leveraged_RP_Ret']-df['Excess_60_40_Ret'])
    
    

    
    result.loc['Annualized Sharpe Ratio','CRSP stocks'] = sharpe(df['Stock_Excess_Vw_Ret'])
    result.loc['Annualized Sharpe Ratio','CRSP bonds'] = sharpe(df['Bond_Excess_Vw_Ret'])
    result.loc['Annualized Sharpe Ratio','Value-weighted portfolio'] = sharpe(df['Excess_Vw_Ret'])
    result.loc['Annualized Sharpe Ratio','60/40 portfolio'] = sharpe(df['Excess_60_40_Ret'])
    result.loc['Annualized Sharpe Ratio','Unlevered RP'] =sharpe(df['Excess_Unleveraged_RP_Ret'])
    result.loc['Annualized Sharpe Ratio','Levered RP'] = sharpe(df['Excess_Leveraged_RP_Ret'])
    result.loc['Annualized Sharpe Ratio','RP minus value weighted'] = sharpe(df['Excess_Leveraged_RP_Ret']-df['Excess_Vw_Ret'])
    result.loc['Annualized Sharpe Ratio','RP minus 60/40'] = sharpe(df['Excess_Leveraged_RP_Ret']-df['Excess_60_40_Ret'])
    
    
    result.loc['Skewness','CRSP stocks'] = skewness(df['Stock_Excess_Vw_Ret'])
    result.loc['Skewness','CRSP bonds'] = skewness(df['Bond_Excess_Vw_Ret'])
    result.loc['Skewness','Value-weighted portfolio'] = skewness(df['Excess_Vw_Ret'])
    result.loc['Skewness','60/40 portfolio'] = skewness(df['Excess_60_40_Ret'])
    result.loc['Skewness','Unlevered RP'] =skewness(df['Excess_Unleveraged_RP_Ret'])
    result.loc['Skewness','Levered RP'] = skewness(df['Excess_Leveraged_RP_Ret'])
    result.loc['Skewness','RP minus value weighted'] = skewness(df['Excess_Leveraged_RP_Ret']-df['Excess_Vw_Ret'])
    result.loc['Skewness','RP minus 60/40'] = skewness(df['Excess_Leveraged_RP_Ret']-df['Excess_60_40_Ret'])
    
    
    result.loc['Excess Kurtosis','CRSP stocks'] = kur(df['Stock_Excess_Vw_Ret'])
    result.loc['Excess Kurtosis','CRSP bonds'] = kur(df['Bond_Excess_Vw_Ret'])
    result.loc['Excess Kurtosis','Value-weighted portfolio'] = kur(df['Excess_Vw_Ret'])
    result.loc['Excess Kurtosis','60/40 portfolio'] = kur(df['Excess_60_40_Ret'])
    result.loc['Excess Kurtosis','Unlevered RP'] =kur(df['Excess_Unleveraged_RP_Ret'])
    result.loc['Excess Kurtosis','Levered RP'] = kur(df['Excess_Leveraged_RP_Ret'])
    result.loc['Excess Kurtosis','RP minus value weighted'] = kur(df['Excess_Leveraged_RP_Ret']-df['Excess_Vw_Ret'])
    result.loc['Excess Kurtosis','RP minus 60/40'] = kur(df['Excess_Leveraged_RP_Ret']-df['Excess_60_40_Ret'])
    
    
    return result.T

PS2_Q4(Port_Rets)

,Annualized Mean,t-stat,Annualized Standard Deviation,Annualized Sharpe Ratio,Skewness,Excess Kurtosis
CRSP stocks,6.73,3.18,19.09,0.35,0.22,7.68
CRSP bonds,1.41,4.42,2.87,0.49,0.24,4.20
Value-weighted portfolio,3.43,2.61,11.88,0.29,-0.56,4.55
60/40 portfolio,4.60,3.57,11.65,0.40,0.23,7.43
Unlevered RP,2.08,5.12,3.67,0.57,0.10,2.69
Levered RP,6.74,5.12,11.88,0.57,0.10,2.69
RP minus value weighted,3.35,3.48,8.69,0.39,0.56,3.91
RP minus 60/40,2.16,2.30,8.51,0.25,-0.04,4.71


Analysis on the difference between my results and the paper results
 - the mean/t-stats/sd/skewness/kurtosis of the stock is slightly different, resons might be different way of processing delisting and updates of CRSP database. Also, the way of processing dividents/spliting might be different. I think the difference is acceptable
 - also the data for bonds are slightly different. In addition to the reasons mentioned for stock data, I realized that the different risk-free data used might be a reason. I am using the RF_30 calculated from t-bill. riskfree data from other sources might affect the value of excess return. 
 - beacuse the stock and bond returns are slightly different, the value-weighted portfolio data are also slightly different. The 60-40 portfolio, leveraged and unleveraged RP are slightly different as well. I think generally such difference is acceptable.
 - seems like the volatility difference for value-weighted portfolio is large(15.08 VS 11.88). However the sample from the professor's note is around 12, so I assume it is due to the difference of data sources instead of miscalculation
 - the skewness and kurtosis for leveraged RP is different. The reasons are analyzed in Q3. 